# Loans Credit Risk Study

This notebook analyzes a loans dataset, covering data cleaning, feature engineering, visualization, and portfolio analysis.  
Metrics like Expected Loss, Loss Given Default, and others are calculated for the portfolio.

## 1. Setup

### 1.1 Imports

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.stats import gaussian_kde
import plotly.io as pio
import os
os.makedirs('images', exist_ok=True)

### 1.2 Default style

In [ ]:
pd.set_option('display.max_columns', None)

pallete = ['rgb(42,72,108)', 'rgb(45, 105, 165)', 'rgb(77,136,180)', 'rgb(123, 188, 221)', 'rgb(193, 211, 217)']

base_template = pio.templates['plotly_white']
plotly_template = base_template.layout.update(
    title=dict(font=dict(family='Arial Black', size=14, color=pallete[0])),
    xaxis=dict(
        tickfont=dict(size=9, family='Arial Black, monospace', color=pallete[0]),
        title_font=dict(size=11, family='Arial Black, monospace', color=pallete[0])),
    yaxis=dict(title_font=dict(size=11, family='Arial Black, monospace', color=pallete[0]), range=[0, 1]),
    plot_bgcolor='rgb(243,250,253)',
    colorway=px.colors.sequential.ice,
    margin=dict(t=80, b=10, r=10, l=10),
    #font=dict(size=9, family='Arial Black', color=pallete[0]),
)

pio.templates['light_blue_sea'] = go.layout.Template(layout=plotly_template)
pio.templates.default = "light_blue_sea"

W = 600
H = 400



### 1.3 Plot functions

In [ ]:
def default_rate_bar_plot(df, xlabel:str, title, **kwargs):

    df = df.copy()
    df['default_rate'] = df[0] / (df[0]+df[1])
    
    # Create the vertical bar chart
    fig = px.bar(
        df,
        x=df.index,
        y='default_rate',
        orientation="v",
        text='default_rate',
        title=title,
        labels={"default_rate": "Default Rate [%]", df.index.name: xlabel},
        color= df.index,
    )

    fig.update_traces(textposition='outside', texttemplate='%{text:.0%}')
    fig.update_layout(uniformtext_minsize=8)
    fig.update_traces(width=0.6) 
    
    if not kwargs.get('show_color_map', False):
        fig.update_coloraxes(showscale=False)


    # Update layout
    fig.update_layout(
        #barmode="stack",
        showlegend=False,
        yaxis=dict(
            tickformat=".0%",
            range=kwargs.get('yaxis_range', None)
            ),
        width=W,
        height=H,
    )

    return fig

def unvariate_bar_plot(df, col_name:str, title:str, xlabel:str, limit=5, **kwargs):
    
    purpose_counts = df[col_name].value_counts(True).head(limit).reset_index()
    purpose_counts.columns = [col_name, 'count']

    fig = px.bar(
            purpose_counts,
            x=col_name,
            y='count' ,
            title=title,
            labels={col_name: xlabel, 'count': 'Percentage [%]'},
            color=col_name,
            text='count',
        )

        # Update trace settings for text position
    fig.update_traces(
            textposition='outside', 
            texttemplate='%{text:.2%}',
            width=0.6,
        )
        
    # Update layout
    fig.update_layout(
            showlegend=False,
            yaxis=dict(
                tickformat=".0%",
                range=kwargs.get('yaxis_range', [0,1])
                ),
            width=W,
            height=H,
        )
        
    return fig

    

## 2. Data preprocessing

In [156]:
loans_df = pd.read_csv("loan_data_2007_2014.csv", )
loans_df = loans_df.iloc[:,2:] # Ignore bugged indexes
loans_df.describe()

C:\Users\rafae\AppData\Local\Temp\ipykernel_26088\2826051011.py:1: DtypeWarning:

Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.



,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,4.662850e+05,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,4.662810e+05,466285.000000,466256.000000,466256.000000,215934.000000,62638.000000,466256.000000,466256.000000,4.662850e+05,465945.000000,466256.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,466140.000000,98974.000000,466285.0,0.0,0.0,0.0,466256.000000,3.960090e+05,3.960090e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.960090e+05,0.0,0.0,0.0
mean,1.459766e+07,14317.277577,14291.801044,14222.329888,13.829236,432.061201,7.327738e+04,17.218758,0.284678,0.804745,34.104430,74.306012,11.187069,0.160564,1.623020e+04,56.176947,25.064430,4410.062342,4408.452258,11540.686220,11469.892747,8866.014657,2588.677225,0.650129,85.344211,8.961534,3123.913796,0.009085,42.852547,1.0,NaN,NaN,NaN,0.004002,1.919135e+02,1.388017e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.037909e+04,NaN,NaN,NaN
std,1.168237e+07,8286.509164,8274.371300,8297.637788,4.357587,243.485550,5.496357e+04,7.851121,0.797365,1.091598,21.778487,30.357653,4.987526,0.510863,2.067625e+04,23.732628,11.600141,6355.078769,6353.198001,8265.627112,8254.157579,7031.687997,2483.809661,5.265730,552.216084,85.491437,5554.737393,0.108648,21.662591,0.0,NaN,NaN,NaN,0.068637,1.463021e+04,1.521147e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.724713e+04,NaN,NaN,NaN
min,7.047300e+04,500.000000,500.000000,0.000000,5.420000,15.670000,1.896000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,NaN,NaN,NaN,0.000000,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN
25%,4.379705e+06,8000.000000,8000.000000,8000.000000,10.990000,256.690000,4.500000e+04,11.360000,0.000000,0.000000,16.000000,53.000000,8.000000,0.000000,6.413000e+03,39.200000,17.000000,0.000000,0.000000,5552.125349,5499.250000,3708.560000,957.280000,0.000000,0.000000,0.000000,312.620000,0.000000,26.000000,1.0,NaN,NaN,NaN,0.000000,0.000000e+00,2.861800e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.350000e+04,NaN,NaN,NaN
50%,1.194108e+07,12000.000000,12000.000000,12000.000000,13.660000,379.890000,6.300000e+04,16.870000,0.000000,0.000000,31.000000,76.000000,10.000000,0.000000,1.176400e+04,57.600000,23.000000,441.470000,441.380000,9419.250943,9355.430000,6817.760000,1818.880000,0.000000,0.000000,0.000000,545.960000,0.000000,42.000000,1.0,NaN,NaN,NaN,0.000000,0.000000e+00,8.153900e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.280000e+04,NaN,NaN,NaN
75%,2.300154e+07,20000.000000,20000.000000,19950.000000,16.490000,566.580000,8.896000e+04,22.780000,0.000000,1.000000,49.000000,102.000000,14.000000,0.000000,2.033300e+04,74.700000,32.000000,7341.650000,7338.390000,15308.158460,15231.310000,12000.000000,3304.530000,0.000000,0.000000,0.000000,3187.510000,0.000000,59.000000,1.0,NaN,NaN,NaN,0.000000,0.000000e+00,2.089530e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.790000e+04,NaN,NaN,NaN
max,4.086083e+07,35000.000000,35000.000000,35000.000000,26.060000,1409.990000,7.500000e+06,39.990000,29.000000,33.000000,188.000000,129.000000,84.000000,63.000000,2.568995e+

In [45]:
loans_df['risky'] = loans_df['loan_status'].isin(['Fully Paid', 'Current']).astype(int)

loans_df['last_pymnt_d'] = pd.to_datetime(loans_df['last_pymnt_d'], format='%b-%y')
loans_df['issue_d'] = pd.to_datetime(loans_df['last_pymnt_d'], format='%b-%y')
loans_df['last_credit_pull_d'] = pd.to_datetime(loans_df['last_credit_pull_d'], format='%b-%y')
loans_df['last_pymnt_d_year'] = loans_df['last_pymnt_d'].dt.year
loans_df['issue_d_year'] = loans_df['issue_d'].dt.year
loans_df['last_credit_pull_d_year'] = loans_df['last_credit_pull_d'].dt.year

prime_df = loans_df[loans_df['risky'] == 1]
subprime_df = loans_df[loans_df['risky'] == 0]

## 3. Study

### 3.1 Loans

#### 3.1.1 Loans Purpose

In [54]:
fig=unvariate_bar_plot(loans_df, 
                    col_name='purpose',
                    title='Top 5 Most Common Loan Purposes',
                    xlabel='Loan Purpose',
                    )
fig.write_image("images/loans_purposes.png")
fig.show()

#### 3.1.2 Loans Grades

In [ ]:
fig = unvariate_bar_plot(loans_df, 
                   col_name='grade', 
                   title='Loans Grades',
                   xlabel='Grade',
                   limit=None,)


fig.write_image('images/grades.png')
fig.show()


### 3.1.3 Loans Interest Rate Distribution and Drift

In [ ]:
# Function to calculate KDE for plotting
def calculate_kde(data, points=100):
    kde = gaussian_kde(data)
    x = np.linspace(1, 25, points) 
    y = kde(x)
    return x, y

int_prime_before = prime_df[(prime_df['last_credit_pull_d_year'] <= 2009)]['int_rate']
int_risky_before = subprime_df[(subprime_df['last_credit_pull_d_year'] <= 2009)]['int_rate']
int_prime_after = prime_df[(prime_df['last_credit_pull_d_year'] > 2009)]['int_rate']
int_risky_after = subprime_df[(subprime_df['last_credit_pull_d_year'] > 2009)]['int_rate']


# Calculate KDEs for each dataset
prime_before_x, prime_before_y = calculate_kde(int_prime_before)
subprime_before_x, subprime_before_y = calculate_kde(int_risky_before)
prime_after_x, prime_after_y = calculate_kde(int_prime_after)
subprime_after_x, subprime_after_y = calculate_kde(int_risky_after)

# Create the figure with two subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("2008 Interest Rate Distributions (KDE)", "Post-Crisis Interest Rate Distributions (KDE)"),
)

# Left Chart: Interest rate KDE for 2008 data (before crisis)
fig.add_trace(go.Scatter(
    x=prime_before_x, 
    y=prime_before_y, 
    mode='lines', 
    name='Prime', 
    line=dict(color='rgb(4,217,57)', width=2),
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=subprime_before_x, 
    y=subprime_before_y, 
    mode='lines', 
    name='Risky', 
    line=dict(color='rgb(255,72,88)', width=2),
), row=1, col=1)

# Right Chart: Interest rate KDE post-crisis
fig.add_trace(go.Scatter(
    x=prime_after_x, 
    y=prime_after_y, 
    mode='lines', 
    name='Prime', 
    line=dict(color='rgb(4,217,57)', width=2),
    showlegend=False,
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=subprime_after_x, 
    y=subprime_after_y, 
    mode='lines', 
    name='Risky', 
    line=dict(color='rgb(255,72,88)', width=2),
    showlegend=False,
), row=1, col=2)

# Add vertical lines for the mean interest rates
fig.add_trace(go.Scatter(
    x=[int_prime_before.mean(), int_prime_before.mean()],
    y=[0, 1], 
    mode='lines',
    line=dict(color='rgba(4,217,57, 0.4)', dash='dash'),
    showlegend=False,
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[int_risky_before.mean(), int_risky_before.mean()],
    y=[0, 1],
    mode='lines',
    line=dict(color='rgba(255,72,88, 0.4)', dash='dash'),
    showlegend=False,
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[int_prime_after.mean(), int_prime_after.mean()],
    y=[0, 1],
    mode='lines',
    line=dict(color='rgba(4,217,57, 0.4)', dash='dash'),
    showlegend=False,
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=[int_risky_after.mean(), int_risky_after.mean()],
    y=[0, 1],
    mode='lines',
    line=dict(color='rgba(255,72,88, 0.4)', dash='dash'),
    showlegend=False,
), row=1, col=2)

# Update layout with synchronized axes and labels
fig.update_layout(
    title="Interest Rate Distributions (KDE) Before and After the Crisis",
    xaxis=dict(title="Interest Rate (%)", range=[1, 25]),  # Set range and label for left chart
    xaxis2=dict(title="Interest Rate (%)", range=[1, 25]),  # Set range and label for right chart
    yaxis=dict(title="Density", range=[0, 0.2]),
    yaxis2=dict(title="Density", range=[0, 0.2]),
    showlegend=True,
    height=500,
    width=1000,
    legend=dict(
        x=-0.015, y=1.17, 
        orientation='h',
        traceorder='normal',
        xanchor='left',
        yanchor='top',
        title=''
    ),
    margin=dict(t=120)
)

    
fig.write_image("images/interest_dist.png")
# Show the figure
fig.show()


### 3.2 Borrower

#### 3.2.1 Home Ownership

In [ ]:
fig=unvariate_bar_plot(loans_df, 
                    col_name='home_ownership',
                    title='Borrowers Home Ownership',
                    xlabel='Home Ownership',
                    limit=3,
                    
                    )
fig.write_image("images/home_ownership.png")
fig.show()

## 4. Loan Sensitivity

### 4.1 Default Rate by Grade

In [ ]:
grade_default_counts = loans_df.groupby(['grade', 'risky']).size().reset_index(name='count')

grade_default_pivot = grade_default_counts.pivot_table(
    index='grade',
    columns='risky',
    values='count',
    aggfunc='sum',
    fill_value=0
    )

fig = default_rate_bar_plot(
    df=grade_default_pivot,
    xlabel='Grade',
    title='Default rate by Grade',
    yaxis_range=[0, 0.5],
)

fig.write_image("images/default_by_grade.png")
fig.show()


### 4.2 Default Rate by Annual Income 

In [ ]:
bins = [0, 30000, 60000, 90000, 120000, float('inf')]
labels = ['$0 - $30k', '$30k - $60k', '$60k - $90k', '$90k - $120k', '>$120k']


income_default_rate_df = pd.DataFrame()
income_default_rate_df['income_range'] = pd.cut(loans_df['annual_inc'], bins=bins, labels=labels)
income_default_rate_df['risky'] = loans_df['risky']
income_default_rate_df['grade'] = loans_df['grade']

income_default_rate_df = income_default_rate_df.groupby(['income_range', 'risky']).size().reset_index(name='count')

income_default_rate_df = income_default_rate_df.pivot_table(
    index='income_range',
    columns='risky',
    values='count',
    aggfunc='sum',
    fill_value=0
    )

fig = default_rate_bar_plot(
                    df=income_default_rate_df,
                    xlabel='Income',
                    title='Default Rate by Annual Income',
                    yaxis_range=[0, 0.3]
                    )
fig.write_image("images/default_by_income.png")
fig.show()

C:\Users\rafae\AppData\Local\Temp\ipykernel_26088\3024893751.py:10: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### 4.3 Default Rate by Interest Rate

In [ ]:
interest_bins = [5, 10, 15, 20, 25, float('inf')]
interest_labels = ['5% - 10%', '10% - 15%', '15% - 20%', '20% - 25%', '>25%']

int_default_rate_df = pd.DataFrame()
int_default_rate_df['int_range'] = pd.cut(loans_df['int_rate'], bins=interest_bins, labels=interest_labels)
int_default_rate_df['risky'] = loans_df['risky']
int_default_rate_df['grade'] = loans_df['grade']

int_default_rate_df = int_default_rate_df.groupby(['int_range', 'risky']).size().reset_index(name='count')

int_default_rate_df = int_default_rate_df.pivot_table(
    index='int_range',
    columns='risky',
    values='count',
    aggfunc='sum',
    fill_value=0
    )

fig=default_rate_bar_plot(int_default_rate_df,
                    xlabel='Interest',
                    title='Default Rate by Interest',
                    yaxis_range=[0,0.3]
                    )
fig.write_image("images/default_by_interest.png")
fig.show()

C:\Users\rafae\AppData\Local\Temp\ipykernel_26088\180471551.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### 4.4 Default Rate by DTI

In [ ]:
dti_bins = [0, 5, 10, 15, 20, 25, 30, float('inf')]
dti_labels = ['0% - 5%', '5% - 10%', '10% - 15%','15% - 20%', '20% - 25%', '25% - 30%',  '>30%']

dti_default_rate_df = pd.DataFrame()
dti_default_rate_df['dti_range'] = pd.cut(loans_df['dti'], bins=dti_bins, labels=dti_labels)
dti_default_rate_df['risky'] = loans_df['risky']
dti_default_rate_df['grade'] = loans_df['grade']

dti_default_rate_df = dti_default_rate_df.groupby(['dti_range', 'risky']).size().reset_index(name='count')

dti_default_rate_df = dti_default_rate_df.pivot_table(
    index='dti_range',
    columns='risky',
    values='count',
    aggfunc='sum',
    fill_value=0
    )

fig = default_rate_bar_plot(
    df=dti_default_rate_df,
    xlabel='DTI',
    title='Default Rate by Debt To Income Ratio',
    yaxis_range=[0,0.2]
    )
fig.write_image("images/deafults_by_dti.png")
fig.show()

C:\Users\rafae\AppData\Local\Temp\ipykernel_26088\3160568533.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## 5. Portfolio Analysis

In [ ]:
portfolio = pd.Series()

current_portfolio = loans_df[~loans_df['loan_status'].isin([
    'Fully Paid', 
    'Default', 
    'Charged Off',
    'Does not meet the credit policy. Status:Fully Paid',
    'Does not meet the credit policy. Status:Charged Off'
    ])]

defaulted_loans = loans_df[loans_df['loan_status'].isin([
    'Charged Off', 
    'Does not meet the credit policy. Status:Charged Off', 
    'Default', 
    'Late (31-120 days)'
    ])]

### 5.1 Portfolio Risk Evolution

In [ ]:
pull_d_year_ct = pd.crosstab(loans_df['last_credit_pull_d_year'], loans_df['risky'])
pull_d_year_sum = pull_d_year_ct.sum(axis=1).astype(float)
pull_d_year_rate = pull_d_year_ct.div(pull_d_year_sum, axis=0)
pull_d_year_rate.index = np.arange(2007, 2017)

prime_default_rate = pull_d_year_rate[1]
subprime_default_rate = pull_d_year_rate[0]

# Create figure
fig = go.Figure()

# Plot for Prime Borrowers
fig.add_trace(go.Scatter(x=pull_d_year_rate.index, y=prime_default_rate,
                         mode='lines+markers',
                         name='Prime Loans',
                         line=dict(color='rgb(4,217,57)', width=2),
                         fillcolor='rgba(4, 217, 57, 0.5)',
                         #fill='tozeroy',
                         #marker=dict(symbol='diamond', size=8))
                         ))

# Plot for Subprime Borrowers
fig.add_trace(go.Scatter(x=pull_d_year_rate.index, y=subprime_default_rate,
                         name='Risky Loans',
                         line=dict(color='rgb(255,72,88)', width=2),
                         #fill='tozeroy',
                         fillcolor = 'rgba(255,72,88,0.5)',
                         #marker=dict(symbol='diamond', size=8)
                         ))

fig.add_shape(
            type='rect',
            x0=2009,
            x1=2016,
            y0=0,
            y1=1,
            fillcolor='skyblue',
            opacity=0.1
        )


# Add percentage labels
for x, y in zip(pull_d_year_rate.index, prime_default_rate.round(2)):
    fig.add_annotation(x=x, y=y+0.05, text=f'{y*100:.2f}%', showarrow=False, 
                       font=dict(size=10, color='rgb(4,217,57)'),
                       align='center', valign='bottom')

for x, y in zip(pull_d_year_rate.index, subprime_default_rate.round(2)):
    fig.add_annotation(x=x, y=y+0.05, text=f'{y*100:.2f}%', showarrow=False, font=dict(size=10, color='rgb(255,72,88)'),
                       align='center', valign='bottom')


## 2008 Crisis Decoration

# Add legend for crisis period
fig.add_annotation(
            x=2008,
            y=0.8,
            xref='x',
            yref='paper',
            text = "Housing<br>Crisis",
            ax=-40,
            font = dict(size=12,color='rgba(255,51,51,0.8)'),
            arrowcolor = 'rgba(255,51,51,.5)',
            showarrow=True,
            arrowhead=4,
        )

fig.add_shape(
            type='rect',
            x0=2007,
            x1=2009,
            y0=0,
            y1=1,
            fillcolor='red',
            opacity=0.1
        )
fig.add_shape(
            type='line',
            x1=2009,
            x0=2009,
            y0=0,
            y1=1,
            line=dict(color='rgba(255,51,51,.5)'),
        )


# Update layout for aesthetics
fig.update_layout(
    title='Loans Risk Evolution Over Time',
    xaxis_title='Year',
    yaxis_title='Percentage [%]',
    xaxis=dict(tickmode='array', tickvals=np.arange(2007, 2017, 1)),
    legend=dict(
        x=-0.008, y=1.12, 
        orientation='h',
        traceorder='normal',
        xanchor='left',
        yanchor='top',
        title=''
    ),
    width=800,
    height=400,

)

fig.write_image("images/risk_evolution.png")
fig.show()


### 5.2 Max Exposure

The total amount of the loans still outstanding

In [157]:
EAD = current_portfolio['out_prncp'].sum() # TODO Include possible collaterals - current_portfolio[]
portfolio['Exposure at Default'] = f'{(EAD/10e6):.2f}'
print(f'Potfolio EAD: ${(EAD/10e6):.2f}MM')

Potfolio EAD: $204.76MM


### 5.3 Probability of Default (PD)

In [143]:
PD = len(defaulted_loans) / len(current_portfolio)
portfolio['Probability of Default'] = f'{(PD):.2%}'
print(f'Potfolio PD: {(PD):.2%}')

Potfolio PD: 21.64%


### 5.4 Default Recovery Rate

In case of a default, how much of the principal can be recovered

In [144]:
recoveries_from_defaulted = defaulted_loans['recoveries'].sum()
principal_from_defaulted = defaulted_loans['out_prncp'].sum()
RR = recoveries_from_defaulted / principal_from_defaulted
portfolio['Recovery Rate'] = f'{RR:.2%}'
print(f'Portfolio Recovert Rate {RR:.2%}')

Portfolio Recovert Rate 50.98%


### 5.5 Loss Given Default (LGD)

Loss after recovery from defaults

In [145]:
LGD_dollars = EAD * (1 - RR)
LGD_pct = 1 - RR
portfolio['Loss Given Default ($)'] = f'{(LGD_dollars/10e6):.2f}MM'
portfolio['Loss Given Default'] = f'{LGD_pct:.2%}'
print(f'Portfolio Loss Given Default: {LGD_pct:.2%}')

Portfolio Loss Given Default: 49.02%


### 5.6 Expected Loss

The total amount is expected to be lost with defaults

In [146]:
EL = EAD * PD * LGD_pct
portfolio['Expected Loss'] = f'{(EL/10e6):.2f}MM'
print(f'Portfolio Expected Loss: {(EL/10e6):.2f}MM')

Portfolio Expected Loss: 21.72MM


### 5.7 Other metrics

In [154]:
# Weighted Average Interest Rate
wai = ((current_portfolio['int_rate']/100) * current_portfolio['loan_amnt']).sum() / current_portfolio['loan_amnt'].sum()
portfolio['Weighted Average Interest'] = f"{wai:.2%}"
    
# Portfolio Yield
total_interest_received = current_portfolio['total_rec_int'].sum()
portfolio['Portfolio Yield'] = f"{(total_interest_received / current_portfolio['loan_amnt'].sum()):.2%}"

# Median DTI (current portfolio)
portfolio['Median DTI'] = f"{current_portfolio['dti'].median()}"

# Loan Status Distribution (entire dataset)
#portfolio['Loan Status Distribution'] = loans_df['loan_status'].value_counts(normalize=True).to_dict()

# Average Loan Amount (current portfolio)
#summary['Average Loan Amount'] = current_portfolio['funded_amnt'].mean()

# Concentration Risk (current portfolio)
#summary['Grade Concentration'] = current_portfolio['grade'].value_counts(normalize=True).to_dict()
#summary['Purpose Concentration'] = current_portfolio['purpose'].value_counts(normalize=True).to_dict()


In [155]:
portfolio

Exposure at Default            204.76
Probability of Default         21.64%
Recovery Rate                  50.98%
Loss Given Default ($)       100.37MM
Loss Given Default             49.02%
Expected Loss                 21.72MM
Weighted Average Interest      14.18%
Portfolio Yield                20.27%
Median DTI                       17.7
dtype: object